In [2]:
import requests
import re
from bs4 import BeautifulSoup

In [3]:
response = requests.get(
    "https://attack.mitre.org/techniques/enterprise/")
soup = BeautifulSoup(response.text, "html.parser")
# print(soup.prettify())  

In [4]:
soup_techniques = soup.find_all("tr", class_="technique")

In [158]:
len(soup_techniques)

594

In [159]:
# {"T1057":["xxxx","cccc"]}
techniques = {}
# {"T1057":["xxxx","cccc"]}
sub_techniques_links = {}

In [ ]:
technique_id  = ""
for soup_technique in soup_techniques:
    temp_id = soup_technique.select("td")[0].text.strip()
    if temp_id != "":
        technique_id = temp_id
        technique_description = soup_technique.select("td")[2].text.strip()
        link = soup_technique.select("td")[1].find("a").get("href")
        techniques[technique_id] = [technique_description]
        sub_techniques_links[technique_id] = [link]
    else:
        # technique_id = soup_technique.select("td")[1].text.strip()
        technique_description = soup_technique.select("td")[3].text.strip()
        link = soup_technique.select("td")[2].find("a").get("href")
        techniques[technique_id].append(technique_description)
        sub_techniques_links[technique_id].append(link)

In [161]:
len(techniques)

193

In [ ]:
for key in techniques:
    print(key, techniques[key])

In [ ]:
for link in sub_techniques_links:
    print(sub_techniques_links[link])

In [164]:
from tqdm import tqdm

def clean_text(text):
    pattern = re.compile(r'\[\d+\]')
    text = re.sub(pattern, '', text)
    return text

host = "https://attack.mitre.org/"
for links in tqdm(sub_techniques_links):
    for link in sub_techniques_links[links]:
        response = requests.get(host+link)
        soup = BeautifulSoup(response.text, "html.parser")
        procedure_exist = soup.find(id="examples")
        if procedure_exist is not None:
            next_node = procedure_exist.find_next()
            rows = next_node.find_all("tr")
            for row in rows:
                if row.find("th") is not None:
                    continue
                cells = row.find_all("td")
                procedure = cells[2].text.strip()
                procedure = clean_text(procedure)
                techniques[links].append(procedure)


100%|██████████| 193/193 [14:24<00:00,  4.48s/it]


In [165]:
techniques["T1548"]

['Adversaries may circumvent mechanisms designed to control elevate privileges to gain higher-level permissions. Most modern systems contain native elevation control mechanisms that are intended to limit privileges that a user can perform on a machine. Authorization has to be granted to specific users in order to perform tasks that can be considered of higher risk. An adversary can perform several methods to take advantage of built-in control mechanisms in order to escalate privileges on a system.',
 'An adversary may abuse configurations where an application has the setuid or setgid bits set in order to get code running in a different (and possibly more privileged) user’s context. On Linux or macOS, when the setuid or setgid bits are set for an application binary, the application will run with the privileges of the owning user or group respectively. Normally an application is run in the current user’s context, regardless of which user or group owns the application. However, there are 

In [166]:
import pickle

with open("techniques.pkl", "wb") as f:
    pickle.dump(techniques, f)

In [197]:
import pandas as pd
df = pd.DataFrame(columns=["text","technique_id"])
c = 0
for i in techniques:
    for j in techniques[i]:
        df.loc[c] = pd.Series({"text":j,"technique_id":i})
        c+=1

In [199]:
df

,text,technique_id
0,Adversaries may circumvent mechanisms designed...,T1548
1,An adversary may abuse configurations where an...,T1548
2,Adversaries may bypass UAC mechanisms to eleva...,T1548
3,Adversaries may perform sudo caching and/or us...,T1548
4,Adversaries may leverage the AuthorizationExec...,T1548
...,...,...
12001,Adversaries may bypass application control and...,T1220
12002,Astaroth executes embedded JScript or VBScript...,T1220
12003,Cobalt Group used msxsl.exe to bypass AppLocke...,T1220
12004,Higaisa used an XSL file to run VBScript code.,T1220


In [202]:
df.to_excel("techniques.xlsx", index=False)